Sama seperti konsep pada Tabular Playground Series seri bulan sebelumnya, notebook untuk seri bulan Maret ini juga akan dibuat sesederhana mungkin, mengikuti skema yang ada di peta pikiran [berikut](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html). Kali ini, permasalahan yang dihadapi adalah pengelompokan atau klasifikasi dengan jumlah observasi melampaui 100 ribu. Untuk itu, saya akan menggunakan estimator SGDClassifier.

In [ ]:
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import PowerTransformer, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/train.csv').drop(columns = 'id')
test = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/test.csv').drop(columns = 'id')
submission = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/sample_submission.csv')

Secara umum, dataset train perlu diperiksa apakah mengandung data ganda (duplikat), maupun missing values. Saya sudah memeriksa dengan train.duplicated() dan train.isna().sum(), tidak ditemukan data ganda pada kedua dataset, tetapi ditemukan missing values pada peubah cat10 di dalam dataset test. Hal ini akan ditangani kemudian.

Dataset train (dan juga test) memiliki 30 peubah yang akan digunakan untuk menebak target berupa kelas biner (0 atau 1). Sebagai catatan, peubah id tidak diperhitungkan sebagai peubah masukan karena dianggap hanya berfungsi untuk mengidentifikasi masing-masing observasi saja.

Di antara 30 peubah tersebut, 19 di antaranya bersifat kategorikal, sedangkan sisanya numerikal. Khusus untuk 19 peubah kategorikal, tidak ada keterangan tambahan yang memisahkan mana peubah nominal, mana peubah ordinal. Kali ini, saya mengasumsikan bahwa keseluruhan 19 peubah kategorikal tersebut bersifat ordinal. Sebagai contoh penerapan asumsi tersebut untuk peubah cat0, saya menjalankan kode berikut:

In [ ]:
train.pivot_table(index = 'cat0',values = 'target')

Terlihat bahwa jika peubah cat0 bernilai A, ada 33% kemungkinan target bernilai 1, dibandingkan 6% jika peubah cat0 bernilai B. Dari sini, saya mengasumsikan nilai A memiliki tingkatan yang lebih tinggi dibanding B, sehingga nilai A saya sandikan dengan 1, dan nilai B saya sandikan dengan 0. Secara umum, asumsi yang serupa saya terapkan untuk sisa 18 peubah kategorikal lainnya.

Catatan: untuk beberapa peubah yang memiliki nilai yang sangat beragam, terkadang saya melakukan binning, yaitu mengelompokkan beberapa nilai kedalam suatu sandi yang sama.

In [ ]:
cat0 = {'A' : 1,'B' : 0}
cat1 = {'G' : 14,'L' : 13,'H' : 12,'D' : 11,'E' : 10,'O' : 9,'B' : 8,'N' : 7,'M' : 6,'C' : 5,'K' : 4,'F' : 3,'J' : 2,'I' : 1,'A' : 0}
cat2 = {'Q' : 18,'O' : 17,'M' : 16,'L' : 15,'I' : 14,'J' : 13,'F' : 12,'S' : 11,'G' : 10,'H' : 9,'R' : 8,'K' : 7,'A' : 6,'D' : 5,'N' : 4,'C' : 3,'E' : 2,'B' : 1,'U' : 0}
cat3 = {'N' : 12,'K' : 11,'G' : 10,'L' : 9,'F' : 8,'I' : 7,'C' : 6,'J' : 5,'H' : 4,'B' : 3,'E' : 2,'A' : 1,'D' : 0}
cat4 = {'I' : 19,'O' : 18,'L' : 17,'J' : 16,'A' : 15,'H' : 14,'B' : 13,'K' : 12,'M' : 11,'R' : 10,'Q' : 9,'G' : 8,'F' : 7,'S' : 6,'N' : 5,'D' : 4,'P' : 3,'E' : 2,'T' : 1,'C' : 0}
cat5 = {'BJ' : 14,'AB' : 12,'I' : 11,'BI' : 10,'BU' : 10,'AJ' : 8,'K' : 8,'AH' : 8,'C' : 8,'AL' : 8,'X' : 8,'CK' : 7,'BC' : 7,'BH' : 7,'E' : 7,'AM' : 6,'R' : 6,'BL' : 6,'L' : 6,'CE' : 6,'BR' : 6,'CD' : 6,'AT' : 5,'F' : 5,'BD' : 5,'O' : 5,'AQ' : 5,'M' : 5,'P' : 5,'CF' : 4,'BB' : 4,'AA' : 4,'AR' : 4,'AU' : 4,'BT' : 4,'U' : 4,'S' : 4,'AW' : 4,'CC' : 4,'Y' : 4,'V' : 3,'AI' : 3,'CH' : 3,'BA' : 3,'H' : 3,'AE' : 3,'D' : 3,'Q' : 3,'CI' : 3,'CG' : 3,'CJ' : 2,'CL' : 2,'AY' : 2,'BO' : 2,'AS' : 2,'AP' : 2,'BQ' : 2,'AN' : 2,'BV' : 2,'J' : 2,'AX' : 1,'T' : 1,'CA' : 1,'AV' : 1,'G' : 1,'BF' : 1,'AO' : 1,'N' : 1,'AF' : 1,'BS' : 1,'BG' : 1,'A' : 0,'BY' : 0,'BN' : 0,'BK' : 0,'BE' : 0,'CB' : 0,'BX' : 0,'BP' : 0,'BM' : 0,'B' : 0,'AK' : 0,'AG' : 0,'ZZ' : 0}
cat6 = {'M' : 15,'O' : 14,'I' : 13,'G' : 12,'W' : 11,'Q' : 10,'S' : 9,'C' : 8,'E' : 7,'F' : 6,'U' : 5,'A' : 4,'D' : 3,'Y' : 2,'K' : 1,'B' : 0}
cat7 = {'BA' : 15,'AC' : 13,'AF' : 12,'C' : 12,'AL' : 10,'I' : 10,'V' : 8,'M' : 8,'L' : 8,'AH' : 7,'AS' : 7,'K' : 6,'A' : 6,'G' : 6,'J' : 6,'AX' : 6,'AE' : 6,'AQ' : 5,'AR' : 5,'S' : 4,'AD' : 4,'AN' : 4,'AJ' : 4,'W' : 3,'AO' : 3,'AM' : 3,'P' : 3,'E' : 3,'Q' : 3,'N' : 3,'U' : 3,'AV' : 3,'AI' : 3,'AW' : 3,'T' : 3,'B' : 3,'AB' : 2,'O' : 2,'AK' : 2,'AP' : 2,'F' : 2,'AG' : 2,'H' : 2,'X' : 2,'AU' : 1,'R' : 1,'AY' : 1,'AA' : 1,'AT' : 1,'Y' : 1,'D' : 1}
cat8 = {'BD' : 17,'AG' : 14,'F' : 13,'K' : 13,'A' : 11,'AV' : 9,'AO' : 9,'AF' : 8,'H' : 7,'BN' : 7,'AT' : 6,'G' : 6,'BI' : 6,'AB' : 6,'M' : 6,'AH' : 5,'X' : 5,'AU' : 5,'J' : 5,'AN' : 5,'BA' : 5,'U' : 5,'AD' : 5,'AX' : 4,'BO' : 4,'AJ' : 4,'AK' : 4,'BM' : 4,'BH' : 4,'AE' : 3,'BG' : 3,'O' : 3,'Y' : 3,'Q' : 3,'BL' : 3,'S' : 3,'BF' : 3,'AQ' : 3,'N' : 3,'BJ' : 3,'L' : 2,'AW' : 2,'BC' : 2,'BK' : 2,'AP' : 2,'C' : 1,'AS' : 1,'AY' : 1,'AI' : 1,'AM' : 1,'I' : 1,'BB' : 1,'AL' : 0,'V' : 0,'B' : 0,'AR' : 0,'BE' : 0,'T' : 0,'E' : 0,'P' : 0,'AC' : 0}
cat9 = {'V' : 18,'B' : 17,'A' : 16,'N' : 15,'I' : 14,'Q' : 13,'R' : 12,'S' : 11,'C' : 10,'U' : 9,'J' : 8,'F' : 7,'L' : 6,'E' : 5,'O' : 4,'G' : 3,'X' : 2,'W' : 1,'D' : 0}
cat10 = {'CD' : 19,'FR' : 17,'JR' : 17,'FT' : 16,'GB' : 16,'L' : 15,'AC' : 15,'GI' : 14,'JO' : 13,'J' : 13,'EY' : 13,'MV' : 13,'KP' : 13,'V' : 13,'FQ' : 12,'E' : 12,'JX' : 12,'AH' : 12,'AB' : 11,'FI' : 11,'JH' : 11,'AP' : 11,'F' : 11,'DR' : 11,'DE' : 10,'BP' : 10,'GA' : 10,'MC' : 10,'GW' : 10,'FO' : 10,'JY' : 10,'LE' : 10,'G' : 10,'IR' : 10,'GK' : 10,'AG' : 10,'DI' : 10,'HW' : 10,'Y' : 10,'KH' : 10,'HC' : 9,'AT' : 9,'KL' : 9,'M' : 9,'KX' : 9,'LV' : 9,'DS' : 9,'BF' : 9,'BC' : 9,'EI' : 9,'GM' : 9,'FC' : 9,'IO' : 9,'ID' : 9,'EV' : 9,'BB' : 9,'DQ' : 9,'HN' : 9,'EQ' : 9,'CC' : 9,'AY' : 8,'FH' : 8,'JV' : 8,'D' : 8,'FJ' : 8,'BT' : 8,'FB' : 8,'MG' : 8,'EW' : 8,'AM' : 8,'HU' : 8,'KF' : 8,'IA' : 8,'JT' : 8,'AL' : 8,'FV' : 8,'IV' : 7,'KG' : 7,'MF' : 7,'JL' : 7,'MD' : 7,'LX' : 7,'DC' : 7,'MT' : 7,'KC' : 7,'JB' : 7,'HQ' : 7,'EL' : 7,'DY' : 7,'LW' : 7,'HV' : 7,'Q' : 7,'IB' : 6,'GL' : 6,'LD' : 6,'LO' : 6,'CB' : 6,'FX' : 6,'HK' : 6,'JW' : 6,'LU' : 6,'HL' : 6,'LF' : 6,'AK' : 6,'EO' : 6,'JJ' : 6,'MB' : 6,'CS' : 6,'FL' : 6,'CU' : 6,'GU' : 6,'DW' : 6,'S' : 6,'KW' : 6,'HR' : 6,'JA' : 6,'LG' : 6,'HO' : 5,'AA' : 5,'W' : 5,'JD' : 5,'IJ' : 5,'CL' : 5,'AN' : 5,'CY' : 5,'CK' : 5,'MH' : 5,'DO' : 5,'GT' : 5,'IX' : 5,'HM' : 5,'IK' : 5,'BL' : 5,'FP' : 5,'IE' : 5,'AR' : 5,'DP' : 4,'T' : 4,'DJ' : 4,'FM' : 4,'AU' : 4,'FK' : 4,'DV' : 4,'HE' : 4,'FE' : 4,'HA' : 4,'LI' : 4,'BY' : 4,'HG' : 4,'FD' : 4,'FS' : 4,'I' : 4,'HX' : 4,'LY' : 4,'LJ' : 4,'GQ' : 4,'JG' : 4,'GN' : 4,'DH' : 4,'MS' : 4,'IH' : 3,'AS' : 3,'AE' : 3,'JI' : 3,'MJ' : 3,'JK' : 3,'IF' : 3,'EU' : 3,'GS' : 3,'X' : 3,'BQ' : 3,'C' : 3,'EE' : 3,'ES' : 3,'LC' : 3,'GC' : 3,'LB' : 3,'BV' : 3,'R' : 3,'P' : 2,'FN' : 2,'KJ' : 2,'KB' : 2,'LM' : 2,'CG' : 2,'K' : 2,'KA' : 2,'AV' : 2,'KV' : 2,'CW' : 2,'IG' : 2,'KT' : 2,'HJ' : 1,'JP' : 1,'BG' : 1,'HB' : 1,'LL' : 1,'GX' : 1,'LN' : 1,'HP' : 1,'GE' : 1,'EA' : 1,'KR' : 1,'KS' : 1,'KQ' : 1,'FG' : 1,'BI' : 1,'CR' : 1,'CO' : 1,'CI' : 1,'IT' : 1,'EP' : 1,'IC' : 1,'DD' : 0,'HH' : 0,'JN' : 0,'BM' : 0,'LQ' : 0,'CJ' : 0,'O' : 0,'CP' : 0,'AD' : 0,'DF' : 0,'EK' : 0,'EC' : 0,'FA' : 0,'BD' : 0,'GY' : 0,'DU' : 0,'LT' : 0,'LR' : 0,'AF' : 0,'GV' : 0,'EN' : 0,'DX' : 0,'GJ' : 0,'MW' : 0,'MI' : 0,'MU' : 0,'LK' : 0,'BA' : 0,'MR' : 0,'AW' : 0,'MA' : 0,'MQ' : 0,'MP' : 0,'HI' : 0,'ME' : 0,'MO' : 0,'HF' : 0,'ML' : 0,'MK' : 0,'DT' : 0,'AJ' : 0,'GF' : 0,'EB' : 0,'GH' : 0,'CM' : 0,'CN' : 0,'CQ' : 0,'GD' : 0,'FF' : 0,'JE' : 0,'FW' : 0,'JC' : 0,'CT' : 0,'EF' : 0,'IY' : 0,'CX' : 0,'IU' : 0,'ED' : 0,'IQ' : 0,'IP' : 0,'DA' : 0,'IN' : 0,'IM' : 0,'JU' : 0,'EG' : 0,'EH' : 0,'BX' : 0,'LH' : 0,'BO' : 0,'DN' : 0,'GG' : 0,'BS' : 0,'DM' : 0,'IL' : 0,'KU' : 0,'KN' : 0,'GR' : 0,'DL' : 0,'KK' : 0,'DK' : 0,'KI' : 0,'CF' : 0,'HY' : 0,'KD' : 0,'CH' : 0,'JF' : 0}
cat11 = {'B' : 1,'A' : 0}
cat12 = {'A' : 1,'B' : 0}
cat13 = {'B' : 1,'A' : 0}
cat14 = {'B' : 1,'A' : 0}
cat15 = {'D' : 3,'C' : 2,'A' : 1,'B' : 0}
cat16 = {'B' : 3,'C' : 2,'A' : 1,'D' : 0}
cat17 = {'C' : 3,'B' : 2,'D' : 1,'A' : 0}
cat18 = {'D' : 3,'C' : 2,'A' : 1,'B' : 0}

Berikut adalah penerapan skema ordinal encoding untuk menyandikan peubah kategorikal pada dataset train.

In [ ]:
train['cat0'] = train['cat0'].map(cat0)
train['cat1'] = train['cat1'].map(cat1)
train['cat2'] = train['cat2'].map(cat2)
train['cat3'] = train['cat3'].map(cat3)
train['cat4'] = train['cat4'].map(cat4)
train['cat5'] = train['cat5'].map(cat5)
train['cat6'] = train['cat6'].map(cat6)
train['cat7'] = train['cat7'].map(cat7)
train['cat8'] = train['cat8'].map(cat8)
train['cat9'] = train['cat9'].map(cat9)
train['cat10'] = train['cat10'].map(cat10)
train['cat11'] = train['cat11'].map(cat11)
train['cat12'] = train['cat12'].map(cat12)
train['cat13'] = train['cat13'].map(cat13)
train['cat14'] = train['cat14'].map(cat14)
train['cat15'] = train['cat15'].map(cat15)
train['cat16'] = train['cat16'].map(cat16)
train['cat17'] = train['cat17'].map(cat17)
train['cat18'] = train['cat18'].map(cat18)

Berikut adalah penerapan skema ordinal encoding untuk menyandikan peubah kategorikal pada dataset test.

Catatan: pada tahapan ini, missing values pada peubah cat10 juga langsung diisi dengan modus.

In [ ]:
test['cat0'] = test['cat0'].map(cat0)
test['cat1'] = test['cat1'].map(cat1)
test['cat2'] = test['cat2'].map(cat2)
test['cat3'] = test['cat3'].map(cat3)
test['cat4'] = test['cat4'].map(cat4)
test['cat5'] = test['cat5'].map(cat5)
test['cat6'] = test['cat6'].map(cat6)
test['cat7'] = test['cat7'].map(cat7)
test['cat8'] = test['cat8'].map(cat8)
test['cat9'] = test['cat9'].map(cat9)
test['cat10'] = test['cat10'].map(cat10)
test['cat10'] = test['cat10'].fillna(test['cat10'].mode()[0])
test['cat11'] = test['cat11'].map(cat11)
test['cat12'] = test['cat12'].map(cat12)
test['cat13'] = test['cat13'].map(cat13)
test['cat14'] = test['cat14'].map(cat14)
test['cat15'] = test['cat15'].map(cat15)
test['cat16'] = test['cat16'].map(cat16)
test['cat17'] = test['cat17'].map(cat17)
test['cat18'] = test['cat18'].map(cat18)

Kemudian, untuk memperingan beban komputasi, saya mengubah tipe data hasil penyandian peubah kategorikal dari int64 menjadi int8. 

In [ ]:
train[['cat0','cat1','cat2','cat3','cat4','cat5','cat6','cat7','cat8','cat9','cat10','cat11','cat12','cat13','cat14','cat15','cat16','cat17','cat18','target']] = train[['cat0','cat1','cat2','cat3','cat4','cat5','cat6','cat7','cat8','cat9','cat10','cat11','cat12','cat13','cat14','cat15','cat16','cat17','cat18','target']].astype('int8')
test[['cat0','cat1','cat2','cat3','cat4','cat5','cat6','cat7','cat8','cat9','cat10','cat11','cat12','cat13','cat14','cat15','cat16','cat17','cat18']] = test[['cat0','cat1','cat2','cat3','cat4','cat5','cat6','cat7','cat8','cat9','cat10','cat11','cat12','cat13','cat14','cat15','cat16','cat17','cat18']].astype('int8')

Di sisi lain, tipe data peubah numerikal saya ubah dari float64 menjadi float32. 

In [ ]:
train[['cont0','cont1','cont2','cont3','cont4','cont5','cont6','cont7','cont8','cont8','cont9','cont10']] = train[['cont0','cont1','cont2','cont3','cont4','cont5','cont6','cont7','cont8','cont8','cont9','cont10']].astype('float32')
test[['cont0','cont1','cont2','cont3','cont4','cont5','cont6','cont7','cont8','cont8','cont9','cont10']] = test[['cont0','cont1','cont2','cont3','cont4','cont5','cont6','cont7','cont8','cont8','cont9','cont10']].astype('float32')

In [ ]:
X = train.drop(columns = 'target')
y = train['target']

Jika kita perhatikan, kelas pada peubah target tidaklah seimbang, di mana perbandingan kelas 0 dan 1 kurang lebih sekitar 3 : 1. Hal ini dapat diperiksa dengan menjalankan kode y.value_counts(). Untuk menyeimbangkan kelas, saya menggunakan method SMOTE [1].

Catatan: Jika tertarik, Anda dapat mencoba strategi resampling yang lain sebagaimana disuguhkan pada tautan [ini](https://imbalanced-learn.org/stable/references/index.html).

In [ ]:
X,y = SMOTE(random_state = 42).fit_resample(X,y)

Kemudian, karena saya menggunakan estimator berbasis model linear, saya berpikir bahwa distribusi peubah perlu diubah agar sedekat mungkin dengan distribusi Gaussian. Sebelumnya, tentu saja kita dapat memeriksa kecondondan data (skewness) dengan kode X.skew(). Untuk melakukan alihragam distribusi, saya menggunakan method PowerTransformer [2].

Setelahnya, menurut saya juga penting untuk melakukan penskalaan (scaling), dengan asumsi untuk menhindari bias terhadap peubah bernilai besar. Untuk itu, saya menggunakan method MinMaxScaler.

In [ ]:
columns = ['cat1','cat2','cat3','cat4','cat5','cat6','cat7','cat8','cat9','cat10','cat15','cat16','cat17','cat18','cont0','cont1','cont2','cont3','cont4','cont5','cont6','cont7','cont8','cont9','cont10']

In [ ]:
transformed = PowerTransformer().fit_transform(X[columns].values)
transformed = MinMaxScaler().fit_transform(transformed)

In [ ]:
for axis, col in enumerate(columns):
    X[col] = pd.Series(transformed[:,axis])

In [ ]:
transformed = PowerTransformer().fit_transform(test[columns].values)
transformed = MinMaxScaler().fit_transform(transformed)

In [ ]:
for axis, col in enumerate(columns):
    test[col] = pd.Series(transformed[:,axis])

Setelah dataset train (X) maupun test saya anggap tuntas dikondisikan, langkah selanjutnya adalah masuk ke tahapan pemodelan.

Pengelompokan atau klasifikasi dilakukan menggunakan SGDCLassifier() dengan cross validation untuk menguji keumuman model. Model dievaluasi dengan area under receiver operating characteristic curve sebagaimana metrics yang digunakan oleh platfom kaggle untuk kompetisi ini.

Catatan: agar scoring metric roc_auc dapat digunakan, estimator SGDClassifier perlu menggunakan fungsi rugi-rugi (loss function) modified huber.

In [ ]:
clf = SGDClassifier(loss = 'modified_huber',random_state = 42)
scores = cross_val_score(clf,X,y,scoring = 'roc_auc')
print(f'Rerata skor roc auc {scores.mean()} dengan simpangan baku {scores.std()}.')

Setelah model dirasa memiliki keumuman yang layak, model dapat dilatih untuk keseluruhan dataset train.

In [ ]:
clf = clf.fit(X,y)
submission['target'] = pd.Series(clf.predict_proba(test)[:,1])

In [ ]:
submission.set_index('id').to_csv('submission.csv')

Kualitas prediksi dengan teknik seperti di atas ternyata hanya menghasilkan skor roc auc sebesar 0.86655 saja.

Referensi

[1] N. V. Chawla, K. W. Bowyer, L. O.Hall, W. P. Kegelmeyer, “SMOTE: synthetic minority over-sampling technique,” Journal of artificial intelligence research, 321-357, 2002.

[2]In‐Kwon Yeo, Richard A. Johnson, A new family of power transformations to improve normality or symmetry, Biometrika, Volume 87, Issue 4, December 2000, Pages 954–959.